In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.8.2'

# Softmax classification

* 임의의 Dataset 준비
* 3개의 클래스로 분류할 데이터 준비


In [ ]:
x_data = [[1., 2., 1., 1.],
          [2., 1., 3., 2.],
          [3., 1., 3., 4.],
          [4., 1., 5., 5.],
          [1., 7., 5., 5.],
          [1., 2., 5., 6.],
          [1., 6., 6., 6.],
          [1., 7., 7., 7.]] # 8x4
y_data = [[0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 1., 0.],
          [0., 1., 0.],
          [0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.]] # 8x3

x_test = [[1., 1., 1., 1.]]
y_test = [[0., 0., 1.]]


## 임의의 Data를 이용해서 3개의 클래스를 가지는 데이터셋 생성

In [ ]:
#dataset을 선언합니다.
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
dataset = dataset.repeat(1).batch(8)

nb_classes = 3 # class의 개수입니다.

print(tf.Variable(x_data))
print(tf.Variable(y_data))

<tf.Variable 'Variable:0' shape=(8, 4) dtype=float32, numpy=
array([[1., 2., 1., 1.],
       [2., 1., 3., 2.],
       [3., 1., 3., 4.],
       [4., 1., 5., 5.],
       [1., 7., 5., 5.],
       [1., 2., 5., 6.],
       [1., 6., 6., 6.],
       [1., 7., 7., 7.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(8, 3) dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)>


In [ ]:
#Weight and bias setting
W = tf.Variable(tf.random.normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')
variables = [W, b]

tf.print(W,b)

[[-0.242907062 1.42283261 -0.520679533]
 [-0.910576344 0.764537513 -0.205278069]
 [0.126286641 0.545951366 -0.193658948]
 [-1.78183401 1.73435283 -0.686472416]] [0.447182775 0.609918892 -1.53257215]


# 가설 설정

* 가설에서 예측한 값들을 이용해 예측값들을 확률로 표현한다.

## $$ y_k = \frac{exp(x_k)}{\sum_{i=1}^{n}exp(x_i)}  $$

In [ ]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis_softmax(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

tf.print(hypothesis_softmax(tf.Variable(x_data)))

[[0.000110028879 0.999787629 0.00010243502]
 [1.42591136e-06 0.999997735 7.83249277e-07]
 [2.37985881e-10 1 8.85406248e-10]
 ...
 [4.75623718e-13 1 2.94408421e-11]
 [3.84623579e-16 1 2.90402115e-13]
 [1.40677438e-18 1 4.66933e-15]]


## 가설을 검증할 Cross Entropy 함수를 정의합니다

$$
\begin{align}
cost(h(x),y) & = −\sum_{n=1}^{n} Y log(h(x))
\end{align}
$$

In [ ]:
def loss_fn(hypothesis, labels):
    loss = tf.keras.losses.categorical_crossentropy(labels, hypothesis)
    # tf.keras.losses.binary_crossentropy(labels, hypothesis) # 이진분류용 CE Loss
    return loss 

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.01)

### 학습 진행

In [ ]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      loss_value = loss_fn(hypothesis_softmax(features),labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, tf.reduce_mean(loss_value.numpy()).numpy()))

Iter: 0, Loss: 8.9500
Iter: 100, Loss: 4.3541
Iter: 200, Loss: 4.2515
Iter: 300, Loss: 4.2051
Iter: 400, Loss: 4.1770
Iter: 500, Loss: 4.1575
Iter: 600, Loss: 4.1429
Iter: 700, Loss: 4.1313
Iter: 800, Loss: 4.1219
Iter: 900, Loss: 4.1141
Iter: 1000, Loss: 4.1074
Iter: 1100, Loss: 4.1017
Iter: 1200, Loss: 4.0967
Iter: 1300, Loss: 4.0924
Iter: 1400, Loss: 4.0885
Iter: 1500, Loss: 4.0851
Iter: 1600, Loss: 4.0820
Iter: 1700, Loss: 4.0793
Iter: 1800, Loss: 4.0768
Iter: 1900, Loss: 4.0745
Iter: 2000, Loss: 4.0724
Iter: 2100, Loss: 4.0705
Iter: 2200, Loss: 4.0688
Iter: 2300, Loss: 4.0672
Iter: 2400, Loss: 4.0657
Iter: 2500, Loss: 4.0643
Iter: 2600, Loss: 4.0631
Iter: 2700, Loss: 4.0619
Iter: 2800, Loss: 4.0608
Iter: 2900, Loss: 4.0597
Iter: 3000, Loss: 4.0587
Iter: 3100, Loss: 4.0578
Iter: 3200, Loss: 4.0570
Iter: 3300, Loss: 4.0561
Iter: 3400, Loss: 4.0554
Iter: 3500, Loss: 4.0547
Iter: 3600, Loss: 4.0540
Iter: 3700, Loss: 4.0533
Iter: 3800, Loss: 4.0527
Iter: 3900, Loss: 4.0521
Iter: 4000, 

## Sample 데이터를 넣고 테스트해봅시다.

In [ ]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis_softmax(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[1.4749667e-04 3.8588375e-02 9.6126413e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


## 데이터를 이용해서 예측

In [ ]:
b = hypothesis_softmax(x_test)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_test, 1)) # matches with y_data


tf.Tensor([[2.4037788e-04 9.9781770e-05 9.9965990e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)


In [ ]:
def accuracy_fn(hypothesis, labels):
    hypothesis = tf.argmax(hypothesis)
    hypothesis = tf.cast(hypothesis, dtype=tf.float32)
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    labels = tf.argmax(labels)
    labels = tf.cast(labels, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy
  
test_acc = accuracy_fn(hypothesis_softmax(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 1.0000
